# Scratch Pad

In [1]:
# Import packages.
import gzip
import numpy as np
import pandas as pd

## EPO Ancestral Sequence vs Hg19 Reference Sequence

In [2]:
# Define a function to load a epo ancestral sequence.
def load_epo_seq(chrom):
    """
    Returns the EPO ancestral sequence for a focal chromosome.
    """
    # Intialize the ancestral sequence.
    anc_seq = ''
    # Open the fasta file.
    with open(f'../data/homo_sapiens_ancestor_GRCh37_e71/homo_sapiens_ancestor_{chrom}.fa', 'r') as fa_data:
        # Compile the ancestral sequence.
        anc_seq = [line.strip().replace(' ', '') for line in fa_data if not line.startswith('>')]
    return ''.join(anc_seq)

# Define a function the load the hg19 reference sequence.
def load_hg19_seq(chrom):
    """
    Returns the Hg19 reference sequence for a focal chromosome.
    """
    # Intialize a flag for compiling the reference sequence.
    compile_seq = False
    # Intialize a list for storing reference sequence.
    ref_seq = []
    # Open the fasta file.
    with gzip.open(f'../data/hg19/hg19.fa.gz', 'rt') as fa_data:
        # For every line in the fasta file.
        for line in fa_data:
            # Clean the line.
            line = line.strip()
            # If this is a header line.
            if line.startswith('>'):
                # Check to see if this is the focal chromosome.
                if line.lstrip('>') == f'chr{chrom}':
                    # Start compiling the reference sequence.
                    compile_seq = True
                # Else-if this is the next chromosome's header line. 
                elif compile_seq:
                    # Stop iterating after compiling the refrence sequence.
                    break
            # Else-if we are iterating through the focal chromosome.
            elif compile_seq:
                    # Update the reference sequence.
                    ref_seq.append(line)
    return ''.join(ref_seq)

# Define a function to build a EPO alignment dictionary.
def build_epo_aln_dicc(chrom):
    """
    Build the EPO alignment dictionary for a specified chromosome.
    """
    # Intialize a set of alleles to skip.
    skip_set = {'N', '-', '.'}
    # Intialize a dictionary to store the EPO alignment.
    epo_aln_dicc = {}
    # Load the Hg19 and EPO sequences.
    hg19_seq = load_hg19_seq(chrom)
    epo_seq = load_epo_seq(chrom)
    
    # Iterate through the chromosome.
    for pos, (hg19, epo) in enumerate(zip(hg19_seq, epo_seq), start=1):
        # If both alleles are valid.
        if hg19 not in skip_set and epo not in skip_set:
            # Update the dictionary.
            epo_aln_dicc[pos] = (hg19.upper(), epo.upper())
    return epo_aln_dicc

In [3]:
# For every chromosome.
for chrom in [i for i in range(1, 23)]+['X', 'Y']:
    # Load the hg19 and ancestral sequence.
    hg19_seq = load_hg19_seq(chrom)
    epo_seq = load_epo_seq(chrom)
    # Sanity check.
    hg19_bp, epo_bp = len(hg19_seq), len(epo_seq)
    print(f'Chr{chrom}: Hg19 = {hg19_bp}bp; EPO = {epo_bp}bp; QC\'ed = {hg19_bp == epo_bp}')

Chr1: Hg19 = 249250621bp; EPO = 249250621bp; QC'ed = True
Chr2: Hg19 = 243199373bp; EPO = 243199373bp; QC'ed = True
Chr3: Hg19 = 198022430bp; EPO = 198022430bp; QC'ed = True
Chr4: Hg19 = 191154276bp; EPO = 191154276bp; QC'ed = True
Chr5: Hg19 = 180915260bp; EPO = 180915260bp; QC'ed = True
Chr6: Hg19 = 171115067bp; EPO = 171115067bp; QC'ed = True
Chr7: Hg19 = 159138663bp; EPO = 159138663bp; QC'ed = True
Chr8: Hg19 = 146364022bp; EPO = 146364022bp; QC'ed = True
Chr9: Hg19 = 141213431bp; EPO = 141213431bp; QC'ed = True
Chr10: Hg19 = 135534747bp; EPO = 135534747bp; QC'ed = True
Chr11: Hg19 = 135006516bp; EPO = 135006516bp; QC'ed = True
Chr12: Hg19 = 133851895bp; EPO = 133851895bp; QC'ed = True
Chr13: Hg19 = 115169878bp; EPO = 115169878bp; QC'ed = True
Chr14: Hg19 = 107349540bp; EPO = 107349540bp; QC'ed = True
Chr15: Hg19 = 102531392bp; EPO = 102531392bp; QC'ed = True
Chr16: Hg19 = 90354753bp; EPO = 90354753bp; QC'ed = True
Chr17: Hg19 = 81195210bp; EPO = 81195210bp; QC'ed = True
Chr18: Hg1

## Legacy Code vs New Code for Parsing a `.maf.gz` File

In [4]:
### Legacy Code From https://github.com/David-Peede/chimp_maf_to_vcf/chimp_maf_parser.py ###


def maf_reader(maf_file):
    """
    Reads in a .maf file and appends all lines in a synteny sequence alignment
    block to a list which will be parsed by the function maf_parser.
    """
    line = maf_file.readline()
    while line[0] != "a":
        line = maf_file.readline()
    block = []
    while line != "":
        line = maf_file.readline()
        if line == "" or line[0] == "a":
            yield block
            block = []
        elif line[0] == "s": block.append(line)

def maf_parser(maf_block):
    """
    Takes the output from the maf_reader function and creates a dictionary for
    each synteny sequence alignment block where the sequence name are the keys
    and the values are a dictionary consisting of the sequence start, size,
    strand oreintation, length, and the actual sequence.
    """
    output = {}
    for line in maf_block:
        source, start, size, strand, srcSize, seq = line.split()[1:]
        output[source] = {
                'start': int(start),
                'size': int(size),
                'strand': strand,
                'srcSize': int(srcSize),
                'seq': seq
                }
    return output

def extract_calls(maf_file,
                  ref_species='hg19',
                  contig='1',
                  min_length=1):
    """
    maf_file = bgziped .maf file (str)
    ref_species = reference source for the .maf file (str)
    contig = chromosome to extract allele calls for (str)
    min_length = minimum alignment length required to extract calls (int)
    ----------
    Outputs a .txt file to stdout in the format of:
    conting    pos    ref    target
    """
    chromosome = 'chr'+contig
    reference = ref_species+'.'+chromosome
    infile = gzip.open(maf_file, 'rt')
    # outfile = sys.stdout (Don't need this for testing.)
    allele = str.maketrans('-acgtn', 'NACGTN')
    blocks = maf_reader(infile)
    maf_dicc = {}
    for block in blocks:
        block_info = maf_parser(block)
        sequences = block_info.keys()
        if reference != list(sequences)[0]:
            continue
        elif block_info[reference]['size'] < min_length:
            continue
        else:
            true_len = block_info[reference]['size']
            align_len = len(block_info[reference]['seq'])
            ref_ind = [
                    i for i in range(align_len)
                    if block_info[reference]['seq'][i] != '-'
                    ]
            allele_calls = {}
            positions = range(
                    block_info[reference]['start']+1,
                    block_info[reference]['start']+1 + true_len,
                    )
            allele_calls[reference] = block_info[reference]['seq'].\
                replace('-', '').translate(allele)
            allele_calls[list(sequences)[1]] = block_info[list(sequences)[1]]['seq'].\
                translate(allele)
            for j in range(true_len):
                maf_dicc[positions[j]] = (
                    allele_calls[list(sequences)[0]][j],
                    allele_calls[list(sequences)[1]][ref_ind[j]],
                )
    return maf_dicc

In [5]:
### New Optimized Code ###


# Define a function to extract the sequences in an alignment block from a MAF file.
def maf_aln_block_generator(maf_file):
    """
    Iterates through a MAF file and yields the sequence alignment blocks.
    """
    # Intialize a list to store the sequences in the alignment block.
    aln_block = []
    # Iterate through every line.
    for line in maf_file:
        # If this is the start of a new alignment block.
        if line.startswith('a'):
            # If there are sequences from the previous alignment block.
            if aln_block:
                # Yield the sequences in the current alignment block.
                yield aln_block
                # Reset the list to store the sequences in the alignment block.
                aln_block = []
        # Else-if this a sequence within the alignment block.
        elif line.startswith('s'):
            # Update the alignment block.
            aln_block.append(line.strip())
    # If the last alignment block hasn't been processed.
    if aln_block:
        # Yield the sequences in the final alignment block.
        yield aln_block

# Define a function to compile the sequence fields for sequences in a MAF alignment block.
def compile_seq_field_info(maf_aln_block):
    """
    Extracts the sequence fields (see below) per sequence from a MAF alignment block.
    
    src: Name of the source sequence (alignment.chromosome).
    start: Start of the aligning region in the source sequence (zero-based).
    size: The length of the aligning region in the source sequence (non-dash characters only).
    strand: If "-", then the alignment is to the reverse-complemented source.
    srcSize: The size of the entire source sequence, not just the parts involved in the alignment.
    seq: The nucleotides in the alignment including any insertions ("-").
    """
    # Intialize a dictionary to store the sequence fields.
    seq_fields = {}
    # Iterate through every sequence in the alignment block.
    for seq_line in maf_aln_block:
        # Extract the sequence fields.
        src, start, size, strand, srcSize, seq = seq_line.split()[1:]
        # Update the sequence fields dictionary.
        seq_fields[src] = {
            'start': int(start), 'size': int(size),
            'strand': strand, 'srcSize': int(srcSize), 'seq': seq,
        }
    return seq_fields

# Define a function to build a MAF alignment dictionary.
def build_maf_aln_dicc(maf_file, chrom, aln_block_len_thresh=1):
    """
    Build a MAF alignment dictionary for a specified chromosome.
    
    maf_file: Gzipped .maf file.
    chrom: Chromosome to extract allele calls for.
    aln_block_len_thresh: Minimum alignment block length required to extract allele calls from.
    """
    # Intialize the chromosome id.
    chrom_id = f'chr{chrom}'
    # Intialize the reference sequence source.
    ref_seq_src = f'hg19.{chrom_id}'
    # Translation table to convert bases.
    vcf_trans = str.maketrans('-acgtn', 'NACGTN')
    # Intialize a dictionary to store the MAF alignment.
    maf_aln_dicc = {}
    
    # Open the MAF file.
    with gzip.open(maf_file, 'rt') as infile:
        # Iterate through every alignment block.
        for aln_block in maf_aln_block_generator(infile):
            # Extract the sequence fields.
            seq_fields = compile_seq_field_info(aln_block)
            # Grab the sequence fields for the refernce source.
            ref_seq_fields = seq_fields.get(ref_seq_src)
            
            # If the first sequence is the reference sequence and passes the alignment block length threshold.
            if (ref_seq_src == list(seq_fields.keys())[0]) and (seq_fields[ref_seq_src]['size'] >= aln_block_len_thresh):
                # Grab the sequence fields for the non-refernce source.
                alt_seq_fields = next((val for key, val in seq_fields.items() if key != ref_seq_src), None)
                
                # If the non-refernce source alignment is found.
                if alt_seq_fields:
                    # Grab the reference sequence.
                    ref_seq = ref_seq_fields['seq']
                    # Grab the length of the reference sequence (ie the number of ungapped bases).
                    ref_seq_len = ref_seq_fields['size']
                    # Intialize a generator for the non-gap indicies in the reference sequence alignment.
                    nogap_idx_iter = iter(i for i, base in enumerate(ref_seq) if base != '-')
                    # Generate the genomic positions.
                    positions = range(ref_seq_fields['start'] + 1, ref_seq_fields['start'] + 1 + ref_seq_len)
                    # Extract the the translated sequences.
                    ref_seq_trans = ref_seq.replace('-', '').translate(vcf_trans)
                    alt_seq_trans = alt_seq_fields['seq'].translate(vcf_trans)
                    
                    # Iterate through the reference sequence.
                    for j, pos in enumerate(positions):
                        # Update the dictionary.
                        maf_aln_dicc[pos] = (ref_seq_trans[j], alt_seq_trans[next(nogap_idx_iter)])
    return maf_aln_dicc

In [6]:
# For every chromosome.
for chrom in [i for i in range(1, 23)]+['X', 'Y']:
    # Update the dictionaries.
    old_maf = extract_calls('../data/panTro/hg19.panTro6.synNet.maf.gz', 'hg19', str(chrom), 1)
    new_maf = build_maf_aln_dicc('../data/panTro/hg19.panTro6.synNet.maf.gz', chrom)
    # Sanity check.
    old_equals_new = old_maf == new_maf
    print(f'Chr{chrom}: It is {old_equals_new} that the legacy and new code produce the same results.')

Chr1: It is True that the legacy and new code produce the same results.
Chr2: It is True that the legacy and new code produce the same results.
Chr3: It is True that the legacy and new code produce the same results.
Chr4: It is True that the legacy and new code produce the same results.
Chr5: It is True that the legacy and new code produce the same results.
Chr6: It is True that the legacy and new code produce the same results.
Chr7: It is True that the legacy and new code produce the same results.
Chr8: It is True that the legacy and new code produce the same results.
Chr9: It is True that the legacy and new code produce the same results.
Chr10: It is True that the legacy and new code produce the same results.
Chr11: It is True that the legacy and new code produce the same results.
Chr12: It is True that the legacy and new code produce the same results.
Chr13: It is True that the legacy and new code produce the same results.
Chr14: It is True that the legacy and new code produce the s